In [44]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import Imputer


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
df_train = pd.read_csv("../input/uconn_comp_2018_train.csv")
df_test = pd.read_csv("../input/uconn_comp_2018_test.csv")




# Any results you write to the current directory are saved as output.

['uconn_comp_2018_test.csv', 'uconn_comp_2018_train.csv']


In [ ]:
#data analysis


In [45]:
columnsRequiredTraining = ['claim_number', 'age_of_driver', 'gender', 'marital_status',
       'safty_rating', 'annual_income', 'high_education_ind','address_change_ind',
         'zip_code', 'claim_date',
       'claim_day_of_week', 'accident_site', 'past_num_of_claims',
       'witness_present_ind', 'liab_prct', 'channel',
        'claim_est_payout', 'age_of_vehicle',
        'vehicle_price', 'vehicle_color', 'vehicle_weight',
       'fraud']
columnsRequiredTesting = ['claim_number', 'age_of_driver', 'gender', 'marital_status',
       'safty_rating', 'annual_income', 'high_education_ind','address_change_ind',
         'zip_code', 'claim_date',
       'claim_day_of_week', 'accident_site', 'past_num_of_claims',
       'witness_present_ind', 'liab_prct', 'channel',
        'claim_est_payout', 'age_of_vehicle',
        'vehicle_price', 'vehicle_color', 'vehicle_weight']
TrainingData= df_train[columnsRequiredTraining]
TestingData= df_test[columnsRequiredTesting]

In [4]:
#data cleaning
#values = TrainingData.age_of_vehicle
#imputer = Imputer()
#transformed_values = imputer.fit_transform(values)


TrainingData["age_of_vehicle"]=TrainingData["age_of_vehicle"].fillna(TrainingData["age_of_vehicle"].mean())
TrainingData["claim_est_payout"]=TrainingData["claim_est_payout"].fillna(TrainingData["claim_est_payout"].mean())
TrainingData["marital_status"]=TrainingData["marital_status"].fillna(TrainingData["marital_status"].mean())
TrainingData["witness_present_ind"]=TrainingData["witness_present_ind"].fillna(TrainingData["witness_present_ind"].mean())

TestingData["age_of_vehicle"]=TestingData["age_of_vehicle"].fillna(TestingData["age_of_vehicle"].mean())
TestingData["claim_est_payout"]=TestingData["claim_est_payout"].fillna(TestingData["claim_est_payout"].mean())
TestingData["marital_status"]=TestingData["marital_status"].fillna(TestingData["marital_status"].mean())
TestingData["witness_present_ind"]=TestingData["witness_present_ind"].fillna(TestingData["witness_present_ind"].mean())
TestingData.isnull().sum()
                            

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#index

claim_number           0
age_of_driver          0
gender                 0
marital_status         0
safty_rating           0
annual_income          0
high_education_ind     0
address_change_ind     0
zip_code               0
claim_date             0
claim_day_of_week      0
accident_site          0
past_num_of_claims     0
witness_present_ind    0
liab_prct              0
channel                0
claim_est_payout       0
age_of_vehicle         0
vehicle_price          0
vehicle_color          0
vehicle_weight         0
dtype: int64

In [46]:
X_train= TrainingData.iloc[:,:-1]
X_train= X_train.select_dtypes(exclude=['object'])
X_train
y_train= TrainingData.iloc[:,-1]
y_train
X_test= TestingData.iloc[:,:]
X_test= X_test.select_dtypes(exclude=['object'])
X_test

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# split data into training and validation data, for both predictors and target
# The split is based on a random number generator. Supplying a numeric value to
# the random_state argument guarantees we get the same split every time we
# run this script.
train_x, val_x, train_y, val_y = train_test_split(X_train, y_train,train_size=0.7, 
                                                    test_size=0.3, random_state = 0)


In [47]:
# Handling missing values


from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer

#case 1: Get Model Score from Dropping Columns with Missing Values
#We've loaded a function score_dataset(X_train, X_test, y_train, y_test) to compare the quality of diffrent approaches to missing values. 
#This function reports the out-of-sample MAE score from a RandomForest.


def score_dataset(train_x, val_x, train_y, val_y):
    model = RandomForestRegressor()
    model.fit(train_x, train_y)
    preds = model.predict(val_x)
    return mean_absolute_error(val_y, preds)



cols_with_missing = [col for col in train_x.columns 
                                 if train_x[col].isnull().any()]
reduced_X_train = train_x.drop(cols_with_missing, axis=1)
reduced_X_test  = val_x.drop(cols_with_missing, axis=1)
print("Mean Absolute Error from dropping columns with Missing Values:")
print(score_dataset(reduced_X_train, reduced_X_test, train_y, val_y))


#case 2 Get Model Score from Imputation:- 


my_imputer = Imputer()
imputed_X_train = my_imputer.fit_transform(train_x)
imputed_X_test = my_imputer.transform(val_x)
print("Mean Absolute Error from Imputation:")
print(score_dataset(imputed_X_train, imputed_X_test, train_y, val_y))



Mean Absolute Error from dropping columns with Missing Values:
0.2639444444444445
Mean Absolute Error from Imputation:
0.26522222222222225


In [49]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

# Define model
my_model = DecisionTreeRegressor()
# Fit model
my_model.fit(train_x, train_y)

# get predicted prices on validation data
val_predictions = my_model.predict(val_x)
print(mean_absolute_error(val_y, val_predictions))


xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)
xg_reg.fit(train_x,train_y)
val_predictions = my_model.predict(val_x)
print("XGBoost:")
print(mean_absolute_error(val_y, val_predictions))


forest_model = RandomForestRegressor()
forest_model.fit(train_x, train_y)
my_preds = forest_model.predict(val_x)
print("Random Forest:")
print(mean_absolute_error(val_y, my_preds))


#xg_reg.fit(X_train,y_train)
#preds = xg_reg.predict(X_test)
#print(preds)


  



##rmse = np.sqrt(mean_squared_error(y_test, preds))
#print("RMSE: %f" % (rmse))

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [24]:
forest_model = RandomForestRegressor()
forest_model.fit(train_x, train_y)
my_preds = forest_model.predict(val_x)
print(mean_absolute_error(val_y, my_preds))

0.2607037037037037


In [ ]:
my_submission = pd.DataFrame({'claim_number':df_test.claim_number , 'fraud': preds})
# you could use any filename. We choose submission here
my_submission.to_csv('sampleSubmission.csv', index=False)